In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test, save, load, get_loss_history
from Models.basic_models import average_model, two_layer_basic, post_process
from Models.nm_layer import nm_layer_net

set_figsize(20, 15)

Training directory found, 36 series
Validation directory found, 6 series
Testing directory found, 10 series


In [2]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# "Pixl", "Slice", or "None"
preproc_type = "pixel"

validation = TAVR_3_Frame("__valid", preproc=preproc_type, preload=False)
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc=preproc_type, preload=False)
train_loader = tavr_dataloader(training,batch_size=4, shuffle=True, num_workers=2)


ave_model = average_model()
model = nm_layer_net([4,8],[4,1])
post_proc = post_process(kind=preproc_type).to(device=device)
loss_fn = batch_l2_loss()

# CHANGE TO NAME OF JUPYTER NOTEBOOK
model_name = "Model 22 (2,2 layer+PixNorm) Run 1 [No L2 reg]"

using device: cuda


In [3]:
learning_rate = 1e-3
reg = 0

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg)

In [4]:
LOAD = False
iteration_num = -1

if LOAD:
    load(model_name, iteration_num, model, optimizer)
    loss_history = get_loss_history(model_name)
    model.to(device=device)
    # I don't know why these lines are necessary
    # or even what the hell they do
    # but they are
    if str(device) == 'cuda':
        for state in optimizer.state.values():
            for k, v in state.items():
                state[k] = v.cuda()
else:
    loss_history = None

In [5]:
# If multiple GPU
# model = nn.DataParallel(model)

In [5]:
train(model, post_proc, optimizer, train_loader, val_loader, loss_fn, device, 
         model_name, loss_history,
          epochs=2, print_every=30, print_level=4, lr_decay=1)


Iteration 0, loss = 0.9467, corrected loss = 440.5299
Validation loss 452.0557 over 81 frames
model saved to model_checkpoints/Model 22 (2,2 layer+PixNorm) Run 1 [No L2 reg]/Model 22 (2,2 layer+PixNorm) Run 1 [No L2 reg]-0
conv_a1.weight,   	norm: 2.7701e+00, 	update norm: 1.0381e-02 	Update/norm: 3.7475e-03
conv_a1.bias,   	norm: 2.0651e-01, 	update norm: 1.9978e-03 	Update/norm: 9.6745e-03
conv_b1.weight,   	norm: 3.1860e+00, 	update norm: 1.0385e-02 	Update/norm: 3.2597e-03
conv_b1.bias,   	norm: 2.1978e-01, 	update norm: 1.9980e-03 	Update/norm: 9.0909e-03
conv_a2.weight,   	norm: 4.0621e+00, 	update norm: 2.9073e-02 	Update/norm: 7.1572e-03
conv_a2.bias,   	norm: 1.8250e-01, 	update norm: 2.8267e-03 	Update/norm: 1.5488e-02
conv_b2.weight,   	norm: 4.0669e+00, 	update norm: 2.9264e-02 	Update/norm: 7.1958e-03
conv_b2.bias,   	norm: 2.0046e-01, 	update norm: 2.8259e-03 	Update/norm: 1.4097e-02
conv_ab1.weight,   	norm: 2.7239e+00, 	update norm: 4.1391e-02 	Update/norm: 1.5195e-02


.. 0.8373... 0.6604... 0.7454
Iter 170... 0.6670... 0.6170... 0.7832
Iteration 180, loss = 0.8069, corrected loss = 375.4489
Validation loss 358.2465 over 81 frames
model saved to model_checkpoints/Model 22 (2,2 layer+PixNorm) Run 1 [No L2 reg]/Model 22 (2,2 layer+PixNorm) Run 1 [No L2 reg]-180
conv_a1.weight,   	norm: 2.7370e+00, 	update norm: 1.2199e-03 	Update/norm: 4.4569e-04
conv_a1.bias,   	norm: 2.1279e-01, 	update norm: 3.1239e-04 	Update/norm: 1.4681e-03
conv_b1.weight,   	norm: 3.0946e+00, 	update norm: 2.1284e-03 	Update/norm: 6.8778e-04
conv_b1.bias,   	norm: 2.1964e-01, 	update norm: 3.1111e-04 	Update/norm: 1.4164e-03
conv_a2.weight,   	norm: 4.0632e+00, 	update norm: 6.7941e-03 	Update/norm: 1.6721e-03
conv_a2.bias,   	norm: 1.8067e-01, 	update norm: 3.5996e-04 	Update/norm: 1.9924e-03
conv_b2.weight,   	norm: 3.9801e+00, 	update norm: 4.7328e-03 	Update/norm: 1.1891e-03
conv_b2.bias,   	norm: 1.7540e-01, 	update norm: 2.4408e-04 	Update/norm: 1.3916e-03
conv_ab1.weight,

In [ ]:
# One last test + visualize results on 1 validation sequence

test(model, post_proc, val_loader, loss_fn, device)
test(ave_model, post_proc, val_loader, loss_fn, device)

In [ ]:
val_seq = TAVR_Sequence("__valid", preproc=preproc_type)
with torch.no_grad():
    model.eval()
    ave_frames = [post_proc(val_seq[0][0].to(device=device))]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:].to(device=device), val_seq[0][i][None,:].to(device=device)))
        ave_frames += [post_proc(ave_frame[0][0]),  post_proc(val_seq[0][i].to(device=device))]
    ave_frames += [post_proc(val_seq[0][9].to(device=device))]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)